# Benchmark data

We perform a parsing of the Supplementary Information (SI) of the paper of Head-Gordon (HG).
We build a dictionary that contains information on all the molecules of the HG dataset. Selected information are the type (spin-polarized or non spin-polarized) of the computation for each molecule. Moreover we also extract the values of the statical polarizability obtained for the lda (actually two types: Slater and SPW92), pbe and pbe0. We also collect the results obtained with the reference method of the paper of HG, i.e. the CCSD(T).

In [1]:
from pyexcel_ods import get_data
import json

In [2]:
data = get_data('HG_data.ods')
data =  json.dumps(data)
data = json.loads(data)['Sheet1']

Data are represented as follows

In [3]:
print data[0]

[u'molecule', u'spin_pol', u'lda-Slater', u'lda-SPW92', u'pbe', u'pbe0', u'CCSD(T)']


We build a dictionary that codifies all the relevant information. For each molecule we also parse the related file in the geometries folder and extract the spin polarization value that gives the polarization of the ground state of the molecule. 

In [4]:
dataset = {}
for rows in data[1:]:
    molecule=str(rows[0])
    datamol={}
    #get spin polarization indication
    datamol[str(data[0][1])]=str(rows[1]).lower()
    #get reference data
    ida=2
    for idata in range(2,7):
        datamol[str(data[0][idata])] = rows[ida:ida+3]
        ida+=3
    #get spin polarization value
    f=open('geometries/'+molecule+'.xyz')
    lines=f.readlines()
    datamol['mpol_ref']=lines[1].rstrip('/\n').split(' ')[1]
    dataset[molecule]=datamol
    

For instance

In [7]:
dataset['Na2']

{'CCSD(T)': [30.434, 30.434, 55.115],
 'lda-SPW92': [27.441, 27.441, 48.168],
 'lda-Slater': [30.899, 30.899, 52.387],
 'mpol_ref': '1',
 'pbe': [29.048, 29.048, 49.271],
 'pbe0': [30.15, 30.15, 50.108],
 'spin_pol': 'sp'}

In [6]:
molecule = dataset.keys()
molecule.sort()
#molecule

Add the key with the values of the field intensity

In [7]:
data = get_data('HG_field_intensity.ods')
data =  json.dumps(data)
data = json.loads(data)['Sheet1']

In [8]:
for rows in data[1:-3]:
    dataset[str(rows[0])]['field_int']=rows[1]

In [9]:
dataset['CO']

{'CCSD(T)': [1.753, 1.753, 2.283],
 'field_int': 0.01,
 'lda-SPW92': [1.872, 1.872, 2.358],
 'lda-Slater': [1.993, 1.993, 2.49],
 'mpol_ref': '1',
 'pbe': [1.856, 1.856, 2.363],
 'pbe0': [1.778, 1.778, 2.274],
 'spin_pol': 'nsp'}

Save the dataset as yaml file

In [10]:
import yaml

In [52]:
with open('hg_data.yaml', 'w') as outfile:
    yaml.dump(dataset, outfile, default_flow_style=False)